# 0. Install Dependencies

# import data from json

In [1]:
from RoomModel import *

In [2]:
!pip show keras

Name: keras
Version: 2.11.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: f:\programs\anaconda3\envs\reinforcmentlearning\lib\site-packages
Requires: 
Required-by: 


In [3]:
import json
# Opening JSON file
f = open('SampleData.json')

# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
Room =Room.from_dict(data)
  
# Closing file
f.close()

In [4]:
print(Room.furniture.dimensions)

[[10, 20], [10, 10]]


# 1. Test Random Environment with OpenAI Gym

In [5]:
from Room import *

In [6]:
# RoomEnv(Furniture Dimension ,Doors (doors Dimensions , (Door1 pos, Door2 pos,etc )),Windows(windows Dimensions,(window1 pos, window2 pos,etc )) )

In [7]:
env = RoomEnv(Room.furniture.dimensions,Room.doors.doors_dimensions,Room.doors.doors_positions,Room.windows.windows_dimensions,Room.windows.windows_positions,Room.furniture.keywords, Room.room_dimensions)

In [8]:
# episodes = 10
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
#     switch = False
    
#     while not done:
#         env.render("human")
#         if(switch):
#             action = 0
#             switch=False
#         else:
#             action = 2
#             switch=True
# #         print('CurrentPos:{}'.format(action))
        
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{} '.format(episode, score))
#     env.close()

In [9]:
from rl.agents import DQNAgent
from rl.policy import  BoltzmannQPolicy
from rl.memory import SequentialMemory

# 2. Create a Deep Learning Model with Keras

In [ ]:
from rl.agents import DQNAgent
from rl.policy import  BoltzmannQPolicy
from rl.memory import SequentialMemory
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:

print(states)
print(actions)
print(env.step(0))
print(env.step(0))

In [ ]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=states))  
    model.add(Dense(24, activation='relu', input_shape=states))

    model.add(Dense(24, activation='relu'))
    
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
# del model 

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()
pt_depth = model.layers[0].get_input_shape_at(node_index=0)
print(pt_depth)

# 3. Build Agent with Keras-RL

In [ ]:
print(env.state)


In [ ]:
print(env.furniture_dimensions)

In [ ]:
def build_agent(model, actions):
    policy =  BoltzmannQPolicy()
    memory = SequentialMemory(limit=500000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=300, target_model_update= 1e-3)
    return dqn

In [ ]:
def fitall(steps, dqns):
  for i in range(steps):
    for dqn in dqns:
        dqn.fit(env, nb_steps=steps, visualize=True, verbose=1)

In [ ]:

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn2 = build_agent(model, actions)
dqn2.compile(Adam(lr=1e-3), metrics=['mae'])

fitall(100000,[dqn2,dqn])
env.close(); 

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H%M%S")
dqn.save_weights(model.name+current_time+'dqn_weights.h5f', overwrite=True)

In [ ]:

scores = dqn.test(env, nb_episodes=5, visualize=True)
env.close();
print(scores.history['episode_reward'])

In [ ]:
env.state

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('sequential_1doorWindowRandomStartSuccessdqn_weights2.h5f')

In [ ]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
env.close();
print(scores.history['episode_reward'])

In [ ]:
env.render3()

# 4. Reloading Agent from Memory

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)